# Developing an integration routine.

In this notebook, I develop a struct and methods that 
describe wiener processes, their refinement, and integration with respect to said wiener process.

I then use that to integrate a single stochastic path.

In [1]:
using Random, Plots
import Statistics
using LinearAlgebra

In [2]:
T = 1.0
N = 25000
Δt = T/N

4.0e-5

In [3]:
struct WienerProcess
    #the two key things to know are 
    path::Vector{Float64}
    Δt::Float64
   
    #This creates a wiener process using a given path, while 
    #enforcing the invariant that a WienerProcess starts at zero
    WienerProcess(p,d) = p[1] != 0.0 ? new(vcat(0.0,p),d) : new(p,d)
end

In [4]:
#Adding methods (not as member functions though)
function get_steps(p::WienerProcess)
    #This pulls the steps out of the WienerProcess
    len = length(p.path)
    if p.path[1] != 0.0
        error("test")
    end 
    steps = p.path[2:len] - p.path[1:len-1]
end

function refine(p::WienerProcess)
    #This refines the resolution of the given WienerProcess, and returns another in it's stead
    len = length(p.path)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = p.path[ind]
        arr[2*ind] =  (0.5 * (p.path[ind] + p.path[ind+1]) .+ (((√p.Δt)/2)*randn(1)))[1]
    end
    arr[2*len-1] = p.path[len]
    
    return WienerProcess(arr, p.Δt/2)
end

function spread_walk(walk)
    len = length(walk)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = walk[ind]
        arr[2*ind] =  (0.5 * (walk[ind] + walk[ind+1]) ) 
    end
    arr[2*len-1] = walk[len]
    
    return arr
end

spread_walk (generic function with 1 method)

In [5]:
#Generators
function WienerProcess(size::Int, domain_length::Float64)
    #This function builds a wiener process out of randomly drawn steps
    #it uses the domain length to calculate Δt
    
    #TODO: implement checking for begining with zero
    return WienerProcess(cumsum(randn(size)),domain_length/size)
end

WienerProcess

In [6]:
function integrate(f::Function, wp::WienerProcess)
    #This function integrates across the time domain.
    
    sum = 0
    steps = get_steps(wp)
    
    for i in 1:length(wp.path)-1
        sum += f(wp.path[i]) * (steps[i]) 
    end
    
    return sum
end

integrate (generic function with 1 method)

In [7]:
#Generate a WienerProcess
tp = WienerProcess(N,T)

WienerProcess([0.0, 0.43055729509282836, -0.7551883256384692, -0.15488235684892204, 0.08526395468193038, 0.0418813361132952, 0.028385305967920604, 0.38980755861786465, 0.7722558324029984, -1.1200214451624064  …  99.53928474505717, 100.69761076683456, 98.20521351582244, 96.99536750085274, 96.84828809428465, 96.53642971246676, 95.92159403535307, 96.81594788678467, 97.03105377490182, 97.7088994940777], 4.0e-5)

In [8]:
#test functions
slf(x) = x
unit(x) = 1

unit (generic function with 1 method)

Integrate $f(W_t) = 1$

In [9]:
integrate(unit,tp),tp.path[length(tp.path)]

(97.70889949407767, 97.7088994940777)

In [10]:
rtp = refine(tp)
integrate(unit,rtp),rtp.path[length(rtp.path)]

(97.7088994940777, 97.7088994940777)

# Answering the question given
integrate $f(W_t) = W_t$, refining the partition as you go.

In [11]:
function time_refinements(tp, itt = 10)
    iterated_test = tp
    for iteration in 1:itt
        iterated_test = refine(iterated_test)

        integral = @time integrate(slf,iterated_test)

        path_length = length(iterated_test.path)
        las_value = iterated_test.path[path_length]
        symbolic_integral = 0.5 * (las_value^2 - T )

        println("iteration ",iteration, ". Number of discrete points ", path_length)
        println("\t",integral, " " , symbolic_integral)
        println("\tAbsolute Error: ",abs(integral - symbolic_integral))
        println("\tSquared Error: ",(integral - symbolic_integral)^2)
        println("\tPercentage Error: ",(integral - symbolic_integral)/symbolic_integral, "\n")
    end
end

time_refinements (generic function with 2 methods)

In [12]:
@time time_refinements(tp, 10)

  0.000293 seconds (6 allocations: 1.145 MiB)
iteration 1. Number of discrete points 50001
	-1530.645517748703 4773.014520171889
	Absolute Error: 6303.660037920592
	Squared Error: 3.973612987367704e7
	Percentage Error: -1.320687379281968

  0.000938 seconds (6 allocations: 2.289 MiB)
iteration 2. Number of discrete points 100001
	1621.1856275670054 4773.014520171889
	Absolute Error: 3151.8288926048835
	Squared Error: 9.934025368258927e6
	Percentage Error: -0.6603434536569098

  0.001801 seconds (6 allocations: 4.578 MiB)
iteration 3. Number of discrete points 200001
	3197.099007445472 4773.014520171889
	Absolute Error: 1575.9155127264166
	Squared Error: 2.4835097032517646e6
	Percentage Error: -0.3301719502562217

  0.003297 seconds (6 allocations: 9.156 MiB)
iteration 4. Number of discrete points 400001
	3985.0576746881125 4773.014520171889
	Absolute Error: 787.9568454837763
	Squared Error: 620875.9903447437
	Percentage Error: -0.16508578428866794

  0.007772 seconds (6 allocations: 18

# Finding the expectation and the variance


In [13]:
observed_paths = [WienerProcess(N,T) for i=1:3000];

In [14]:
evaluated_integrals = [integrate(slf,x) for x=observed_paths];

In [15]:
println("Observed mean: ", Statistics.mean(evaluated_integrals)
    ,"\nObserved var: ", Statistics.var(evaluated_integrals))

Observed mean: -136.11883587323365
Observed var: 2.892753990348395e8


In [20]:
#plot(0.0:Δt:T,[x.path for x=observed_paths[1:10]]
#    , legend=false)